In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from pathlib import Path

import pandas as pd
from skimage import io


def load_images(images_path, csv, samples):
    images_list = []
    labels = []
    
    # List all files on the way
    list_files = sorted([os.path.join(images_path, file)
                         for file in os.listdir(images_path) if file.endswith('.png')])
    
    list_files = list_files[: samples]
    
    # Read the csv that contains the information of the images
    df = pd.read_csv(csv)
    
    # Take the image only if it exists in the csv file
    for filename in list_files:
        image_id = int(Path(filename).stem)

        sub_df = df[df["id"] == image_id]
        if sub_df.empty:
            continue

        images_list.append(io.imread(filename))
        labels.append(sub_df.iloc[0]["boneage"])

    return images_list, labels

In [ ]:
from skimage.transform import resize

# Resize images 

def resize_image(images, height, width, anti_aliasing):
    resized_images = []

    for i in images:
        resized_images.append(resize(i, (height, width), anti_aliasing=anti_aliasing))

    return resized_images

In [ ]:
from sklearn.model_selection import train_test_split

# GLOBALS
SAMPLES = 5000

HEIGHT = 28
WIDTH = 28

# Train
train_images_path = "../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset"
train_csv_path = "../input/rsna-bone-age/boneage-training-dataset.csv"

# Load
x, y = load_images(train_images_path, train_csv_path, SAMPLES)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
print("loaded")

# Resize
x_train = resize_image(x_train, HEIGHT, WIDTH, False)
x_test = resize_image(x_test, HEIGHT, WIDTH, False)


# To numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Reshape
x_train = x_train.reshape((x_train.shape[0], 784))
x_test = x_test.reshape((x_test.shape[0], 784))

In [ ]:
# Imports components from Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatte

# Initializes a sequential model
model = Sequential()

# First layer
model.add(Dense(10, activation="relu", input_shape=(784,)))

# Conv
model.add(Conv2D(10, kernel_size=3 activation="relu", input_shape=()))
model.add(Flatten())

# Second layer
model.add(Dense(10, activation="relu"))

# Output layer
model.add(Dense(1, activation="linear"))

# Compile model
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["accuracy", "mean_squared_error"]
)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=3
)

In [ ]:
from matplotlib import pyplot

# Error medio cuadrado
pyplot.plot(history.history['mean_squared_error'])

In [ ]:
from matplotlib import pyplot

y_pred = model.predict(x_test)

fig, ax1 = pyplot.subplots(1,1, figsize = (6,6))
ax1.plot(y_test, y_pred, 'r.', label = 'predictions')
ax1.plot(y_test, y_test, 'b-', label = 'actual')
ax1.legend()
ax1.set_xlabel('Actual Age (Months)')
ax1.set_ylabel('Predicted Age (Months)')


In [ ]:
# # Imports components from Keras
# from keras.models import Sequential
# from keras.layers import Dense, Conv2D, Flatten

# # Initializes a sequential model
# model = Sequential()

# # First layer
# # model.add(Dense(10, activation="relu", input_shape=(784,)))

# # Conv
# model.add(Conv2D(10, kernel_size=3, activation="relu"))
# model.add(Flatten())

# # Second layer
# model.add(Dense(5, activation="relu"))

# # Output layer
# model.add(Dense(1, activation="linear"))

# # Compile model
# model.compile(
#     optimizer="adam",
#     loss="mse",
#     metrics=["accuracy", "mean_squared_error"]
# )